# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [1]:

df_raw  = pd.read_csv('../Data/Cities_clean.csv')

NameError: name 'pd' is not defined

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
# configure the gmaps and putting int the google API key.
# gmaps.configure(api_key=g_key)

# isolate the locations that we want.\
# we want specificall the Lat, Long, and Humidity Data.
test_data  = df_raw[['Lat','Lng','Humidity','Country']]
gmaps.configure(api_key = g_key)



In [63]:
# add the heatmaps according to the test data. 
# to do this, we need to create the heat layer and add it to google maps
# activate google maps 
fig = gmaps.figure()
locations = test_data[["Lat", "Lng"]]
rating = test_data["Humidity"].astype(float)
max_humidity = test_data['Humidity'].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
# max temperature lower than 80 but higher than 70
df_temp = df_raw.loc[df_raw['Max Temp'] < 80 ]
df_temp = df_temp.loc[df_raw['Max Temp'] > 70 ]
# # wind speed less than 10mph
df_wind = df_temp.loc[df_temp['Wind Speed'] < 10]
# # zero cloudiness
df_cloud = df_wind.loc[df_wind['Cloudiness'] == 0]
df_clean = df_cloud.reset_index(drop=True)
df_clean

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,itacare,-14.2775,-38.9967,71.10,89,0,2.82,1627441536,BR
1,mahon,39.8885,4.2658,70.36,88,0,8.05,1627441543,ES
2,altay,47.8667,88.1167,72.52,41,0,7.18,1627440968,CN
3,mareeba,-17.0000,145.4333,78.44,60,0,4.88,1627441568,AU
4,zaysan,47.4667,84.8667,71.20,30,0,8.99,1627441582,KZ
5,tigzirt,36.8886,4.1198,76.66,70,0,7.47,1627441588,DZ
6,milas,37.3164,27.7839,79.95,39,0,4.61,1627441590,TR
7,fez,34.0372,-4.9998,73.65,56,0,3.44,1627441506,MA
8,san lawrenz,36.0553,14.2040,77.86,79,0,4.25,1627441604,MT
9,tukrah,32.5341,20.5791,75.16,77,0,9.44,1627441605,LY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
hotel_df = df_clean.loc[:,['city','Country','Lat','Lng']]
hotel_df['Hotel Name'] = ''
hotel_df

,city,Country,Lat,Lng,Hotel Name
0,itacare,BR,-14.2775,-38.9967,
1,mahon,ES,39.8885,4.2658,
2,altay,CN,47.8667,88.1167,
3,mareeba,AU,-17.0000,145.4333,
4,zaysan,KZ,47.4667,84.8667,
5,tigzirt,DZ,36.8886,4.1198,
6,milas,TR,37.3164,27.7839,
7,fez,MA,34.0372,-4.9998,
8,san lawrenz,MT,36.0553,14.2040,
9,tukrah,LY,32.5341,20.5791,


In [71]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [75]:
import time
import requests

for index, row in hotel_df.iterrows():
    city_name = row.loc['city']
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    print(f'Retriving Reulst for Index {index}: {city_name}')
    try:
        response = requests.get(base_url, params = params).json()
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError,IndexError):
        print("Missing field/result... skipping.")
    print('----------------------------------------------------------------')
    
    time.sleep(1)

print('Finished Searching')

Retriving Reulst for Index 0: itacare
----------------------------------------------------------------
Retriving Reulst for Index 1: mahon
----------------------------------------------------------------
Retriving Reulst for Index 2: altay
----------------------------------------------------------------
Retriving Reulst for Index 3: mareeba
----------------------------------------------------------------
Retriving Reulst for Index 4: zaysan
----------------------------------------------------------------
Retriving Reulst for Index 5: tigzirt
----------------------------------------------------------------
Retriving Reulst for Index 6: milas
----------------------------------------------------------------
Retriving Reulst for Index 7: fez
----------------------------------------------------------------
Retriving Reulst for Index 8: san lawrenz
----------------------------------------------------------------
Retriving Reulst for Index 9: tukrah
-------------------------------------------

In [76]:
hotel_df

,city,Country,Lat,Lng,Hotel Name
0,itacare,BR,-14.2775,-38.9967,Vila Barracuda
1,mahon,ES,39.8885,4.2658,Hotel Son Tretze
2,altay,CN,47.8667,88.1167,阿勒泰市金都酒店
3,mareeba,AU,-17.0000,145.4333,Jackaroo Motel
4,zaysan,KZ,47.4667,84.8667,Zaysan Hotel
5,tigzirt,DZ,36.8886,4.1198,Hôtel Mizrana
6,milas,TR,37.3164,27.7839,Milashan Hotel
7,fez,MA,34.0372,-4.9998,Riad Fes - Relais & Châteaux
8,san lawrenz,MT,36.0553,14.2040,Saint Patrick's Hotel Gozo
9,tukrah,LY,32.5341,20.5791,مصيف مرحبا للعائلات


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))